# Monte Carlo Simulation of the Ising Model

By Alpesh Vyas

## Abstract

In this program I use the Monte Carlo method to simulate the ferromagnetic properties of a two dimensional lattice of spins using Python.  The Ising Model has exact solutions in one dimension and two dimensions, but beyond one dimension analytical analysis becomes difficult.  Using the Monte Carlo method where an inital lattice of randomly oriented spins are flipped until the system comes to an equilibrium I am able to simulate the ferromagnetic properties at various temperatures.

## Background

The Ising Model is a model used to describe ferromagnetism in materials.  Ferromagnetism typically occurs in a material below a critical temperature called the Curie temperature.  Above this temperature, the thermal motion of the ions dominate and the material loses its magnetization.  Below this temperature the material experiences one of three phases: ferromagnetism, ferrimagnetism and antiferromagnetism.  Ferromagnetism is the case where all the spins or magnetic moments align to form a net magnetization.  Ferrimagnetism is the case where there are some spins that are anti-aligned but where there is still a net magnetization.  Lastly, antiferromagnetism is the case where half the spins are aligned in one direction and the other half are aligned in the opposite direction so that they cancel each other to create no net magnetization. 

In its most basic sense it models the interaction between spins in a ferromagnetic material.  For a system that has $N$ sites that contain one spin magnetic moment, each site $S_ij$ can have one of two possible orientations, $S_{ij} = 1$ (spin up) or $S_{ij} = -1$ (spin down).  In a two dimensional lattice configuration each spin site has four nearest neighbors (above, below, left and right) and the Ising Model simplifies the analysis of the ferromagnet by taking into account only the interactions of directly neighboring spins.  The energy of the system is given by the equation below:

$E = -h{\Sum{{s_i}}} -j{\Sum{{s_i}{s_j}}}$

The first term is the interaction of the external magnetic field, $h$ with each individual spin.  The second term is the interaction between each neighboring spin related by a factor $j$.  If there is no external magnetic field present then the energy is simply given by the second term and only takes into account the interactions between neighboring spins.  In this simulation the external magnetic field is taken to be zero.  

## Monte Carlo Method

The Monte Carlo method in analyzing the Ising Model is prety straightforward.  The initial lattice system is in a state of random spin configurations.  We then choose a site on the lattice randomly and calculate the change in energy of the new configuration.  Since we are only concerned with the nearest neighbor interactions we do not need to calculate the energy of the entire system again.  We simply calculate the change in energy since we know which possible values it must take for a given spin flip.

If the change in energy is less than zero then we accept the new spin configuration.  This makes sense since the system will tend to equilibriate to the lowest energy state.  If the change in energy is greater than zero then we accept it with a probability.  Since there is still a chance that the system will go to the higher energy state (however improbable) we must take this into account.  The probability that we use is taken from statistical mechanics and is called the partition function which is given below:

$p=e^{\frac{-{\Delta{E}}}{{k_B}{T}}}$

Where ${\Delta{E}}$ is the change in energy of the system after the spin flip, $T$ is the temperature of the system and $k_B$ is Boltzmann's constant.  In this simulation Boltzmann's constant is set to a value of 1.  In order to utilize this function we generate a random number $x$ between 0 and 1.  If we flip a spin and it ends up having a positive change in energy, we accept this new configuration only if $p>x$.  If $p$ is less than $x$ then we reject the flip and keep the old configuration. 

This process is repeated until all the spins have had a chance to flip at least once. 

## Implementation in Python

In [11]:
from numpy import *
from random import *
from Tkinter import *

The user is allowed to input the number of sites on each row of the two dimensional lattice, the width of each cell in the interface simulation and the number of Monte Carlo steps that are taken in the simulation.  Because the screen on the computer is only so large, I find that keeping the number of sites between 50 and 75 is ideal.  Similarly the width of each cell should be reasonably set.  The width is given in pixels and a width between 10 and 15 is good. Lastly the number of steps in the simulation has one main visual effect on the interface.  If the number of Monte Carlo steps are small then the spins will flip quickly on the interface simulation.  If the number of steps is large then the spins will flip more slowly on the interface.

In [12]:
#defining constants
kb=1.#38*10**-23 #boltzmann

#user inputs
n = input('Input the number of sites in each row: ')
w = input('Input the cell width of each site in pixels:')
steps = input('Input the number of Monte Carlo steps:')

Input the number of sites in each row: 50
Input the cell width of each site in pixels:10
Input the number of Monte Carlo steps:10000


Implementing the energy function in python requires the consideration of the edges of the lattice.  Since the simulation takes into consideration the change in energy based on the neighboring spins, we have to consider the cases where the site on the lattice that is chosen has only two nearest neighbors instead of four.

In [13]:
#defining the change in energy function
def deltaE(i,j):
    if i==0:
        left=lattice[sites-1,j] 
    else:
        left=lattice[i-1,j]
    if i==sites-1:
        right=lattice[0,j]
    else:
        right=lattice[i+1,j]
    if j==0:
        above=lattice[i,sites-1]
    else:
        above=lattice[i,j-1]
    if j==sites-1:
        below=lattice[i,0]
    else:
        below=lattice[i,j+1]
    return 2.0 * lattice[i,j] * (left+right+above+below)

There are a few ways to do visualizations with python such as VPython and the Tkinter module.  VPython requires a separate download and seems to be better for 3D visualizations.  Since Tkinter is already a module that is available in python by importing it, I found it easier.  In many respects Tkinter reminds me of matplotlib when trying to make plots. 

The pixel function below simply looks at a site and assigns it one of two colors.  If the spin at that site is 1 (spin up) then the color assigned is green.  If the spin is equal to -1 (spin down) then the color is grey.  The last line of code in the block below colors in the corresponding pixels on the interface.

In [14]:
#defining a function that will assign a color to the pixel on the user interface for the corresponding site
def pixel(i,j):
    if lattice[i,j]==1:
        pixel = "#2a623d" #slytherin green = spin up
    else:
        pixel = "#aaaaaa" #slytherin grey = spin down
        
    image.put(pixel, to=(i*sitewidth,j*sitewidth,(i+1)*sitewidth,(j+1)*sitewidth))#colors the corresponding pixel in the image


Below is the main Monte Carlo loop.  The code runs through this loop only if the interface is running the simulation. The temperature can be adjusted on the interface (see code further below) and this loop first obtains that temperature.  Then the loop chooses a site at random, it calculates the change in energy by calling the energy function and if that change is less than or equal to zero it flips the spin. If it is greater than zero it flips the spin with a probability. The loop then adjusts the color on the interface.

In [15]:
#monte carlo loop
def montecarlo():
    if running:
        T = tempscale.get()#gets the temperature from the interface
        for step in range(steps):#user defines the number of monte carlo steps
            i = int(random()*sites)#choosing a random row and column
            j = int(random()*sites)
            x=round(uniform(0.,1.),10)#choosing a random number between 0 and 1
            p=exp(-deltaE(i,j)/(kb*T))#partition function
            if deltaE(i,j)<=0:#this flips the spins if the change in energy is less than zero
                lattice[i,j] = -lattice[i,j]
            elif p>x:#if energy is greater than zero, flip the spins only if p is greater than x
                lattice[i,j] = -lattice[i,j]
                pixel(i, j)#calls the pixel function
    window.after(1,montecarlo) #comes back in one ms

In [16]:
sites=n#number of sites in each row
#creating a 2D lattice
lattice = zeros((sites, sites), int)
sitewidth=w#width of each site
canvaswidth=sites * sitewidth#width of canvas
running=False

window=Tk()
window.title("Simulation of the Ising Model")
window.geometry('+50+50')

#creating the canvas to simulate the phase transitions
canvas = Canvas(window, width=canvaswidth, height=canvaswidth)
canvas.pack()
image = PhotoImage(width=canvaswidth, height=canvaswidth)
canvas.create_image((3, 3), image=image, anchor="nw", state="normal")

1

In [17]:
#defining a function that stops and starts the simultation on the interface
def startandstop():
    global running
    running = not running
    if running:
        startbutton.config(text="stop")
    else:
        startbutton.config(text="start")

Below are the main Tkinter commands.  

In [18]:
#this is like the Tkinter equivalent of making graphs in matplotlib
#I had to look up how to do pretty much everything in this section
controlFrame = Frame(window)#creates a frame for everything
controlFrame.pack()
tempscale = Scale(controlFrame, from_=0.01, to=10., resolution=0.01, length=300, orient="horizontal")
tempscale.pack(side="left")
tempscale.set(2.3)#this is the curie temperature of the ising model
templabel = Label(controlFrame, text="temperature:")
templabel.pack(side="left")
spacer = Frame(controlFrame, width=40)
spacer.pack(side="left")
startbutton = Button(controlFrame, text="start", width=8, command=startandstop)
startbutton.pack(side="left")

The last few blocks contain the actual commands that run the code. First a random lattice of spins is created. Then the Monte Carlo function is run followed by the user interface function. 

In [19]:
spins=[-1,1]
#loop that creates a random lattice of spins
for i in range(sites):
    for j in range(sites):
        lattice[i,j]=choice(spins)
        pixel(i,j)#calls pixel function to color lattice accordingly

In [20]:
montecarlo()#runs the monte carlo loop
window.mainloop()#runs the interface that simulates the lattice

## Results

The literature value of the Curie temperature for the Ising Model is roughly T = 2.7 and we can see in the simulation that below this temperature the lattice starts to experience phase transitions where larger regions are aligned either up or down.  Eventually at lower temperatures the lattice becomes completely ferromagnetic and has all the spins aligned in one direction. At higher temperatures above the Curie temperature the thermal motion of the sites become dominant and the lattice systtem loses its magnetization.  This is what is expected from the analysis of the Ising Model. 

Possible extensions of this project include the calculation of other parameters relevant to the subject.  Also, 3D simulations can be done based off of this.  However for a 3D simulation a different module may be necessary such as VPython. 